**Day 5**: ML Workflow 👷 (***live in 1.49/1.50***)

<center><h1 style="color:maroon">Machine Learning Workflow</h1>
    <img src="https://drive.google.com/uc?id=147Lecen6y_grpY_9lWbRj28qay4CBcSN" style="width:1300px">
    <h3><span style="color: #045F5F">Data Science & Machine Learning for Planet Earth Lecture Series</span></h3><h6><i> by Cédric M. John <span style="size:6pts">(2023)</span></i></h6></center>

## Plan for today's Lecture 🗓 

* Motivation for an ML workflow
* Introduction to <code>sklearn.pipeline</code> module
* Writing custom transformers
* Grouping data transformation and models into one object
* Migrating from Notebooks to Python classes

## Intended learning outcomes 👩‍🎓

* Write clean code using pipelines
* Optimize the entire ***data-preparation-to-model-selection*** chain
* Build code deployable locally and on the cloud

# Data Pipelines
<br>

<center><img src="https://drive.google.com/uc?id=1449ihCDJfUk-s9fC3snOpBXI9WHj1oAY" style="width:900px;"><br>
 © Cédric John, 2022; Image generated with <a href="https://openai.com/blog/dall-e/">DALL-E</a><br>
<br>Prompt: Wide angle view of a large metal oil pipeline in the sunrise in the middle of a frozen arctic stepp.</center>

### Dataset

<span style="color:teal">**Todays's dataset:**</span><a href="https://www.kaggle.com/datasets/rohanrao/air-quality-data-in-india"> India air quality data, Kaggle</a><br>
<img src="https://drive.google.com/uc?id=13rbICKbhIfs-K0dKdLxxHybLDMWLHRk2" style="width:900px"/>

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('Lecture_data/India_air_quality_light.csv')

# REMINDER: Always drop duplicates first
data = data.drop_duplicates()

data

,stn_code,state,location,type,so2,no2,date,rainfall,high_level
0,NaN,Maharashtra,Navi Mumbai,Others,12.0,17.0,21/08/2008,69.776897,1
1,334,Gujarat,Vadodara,Industrial,24.0,48.0,02/03/2010,NaN,1
2,NaN,Karnataka,Hubli-Dharwad,Others,NaN,6.8,08/02/2007,79.575206,0
3,NaN,Chandigarh,Chandigarh,Others,NaN,11.0,08/05/2005,108.009090,0
4,598,Karnataka,Bangalore,Sensitive,9.0,19.0,18/02/2014,93.589798,0
...,...,...,...,...,...,...,...,...,...
27865,689,Odisha,Balasore,Residential,2.0,13.0,26/11/2013,NaN,1
27866,665,Himachal Pradesh,Sunder Nagar,Residential,2.0,8.0,07/03/2013,98.072063,0
27867,NaN,Andhra Pradesh,Visakhapatnam,Industrial,11.5,29.1,09/01/2006,79.776776,0
27868,NaN,Kerala,Trivandrum,Others,7.3,27.5,07/04/2008,101.609464,0


## Reminder of the data processing needed on this dataset (Lecture 1):

1. Remove Outliers 
2. Missing Data
3. Scale Features
4. Engineer Features
5. Encode Data
6. Select Feature

* Need to train and apply these to our **training set**

* Need to apply the same transformations to our **test set**

* Need to apply the same transformations to our **new data**

* This implies we need to save (and load) <span style="color:red">**several independant transformers**</span>. 

### <span style="color:teal">Messy, work-intensive and entails large potential for bugs and errors.</span>

##  Pipelines: Chaining Data Transformations
<a href="https://scikit-learn.org/stable/modules/compose.html">Sklearn doc</a><br>

### Principle
* **chains** together multiple steps **in sequence**, e.g.:
* *impute* missing values, *then*
* *scale* numerical features, *then*
* *encode* categorical features, *etc...*

* Make your workflow much easier to read and understand.
* Enforce the implementation and order of steps in your project.
* Make your work reproducible and deployable

<img src="https://drive.google.com/uc?id=13pBbXtuoOyHn8NlZhbsdBVk-kVnib-ep" style="width:1200px">
<a href="https://www.packtpub.com/product/python-machine-learning-third-edition/9781789955750">Rashka, S., 2015, Packt Publishing</a>

## Preprocessing pipelines

All preprocessing steps can be built in a pipeline. We are going to predict whether or not the **pollution level is high** as a function of various features. This will illustrate the use of pipelines.

In [2]:
from sklearn.model_selection import train_test_split

y = data.high_level # Target
X = data.drop(columns='high_level') #Feature

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

X_train

,stn_code,state,location,type,so2,no2,date,rainfall
14606,NaN,Maharashtra,Chandrapur,Others,32.0,52.6,18/02/2008,94.475205
21850,280,Maharashtra,Nashik,Residential,25.0,26.0,25/07/2013,46.851849
23190,581,Andhra Pradesh,Khammam,Residential,2.0,10.0,12/01/2010,88.105338
16552,82,Madhya Pradesh,Nagda,Industrial,32.0,17.0,19/11/2015,71.220319
14739,38,Tamil Nadu,Chennai,Industrial,13.0,13.0,08/12/2014,110.480150
...,...,...,...,...,...,...,...,...
14654,298,Rajasthan,Jaipur,Residential,6.0,30.0,06/05/2014,91.851699
1605,NaN,Karnataka,Mysore,Industrial,11.5,15.9,15/06/2007,NaN
26844,NaN,Tamil Nadu,Chennai,Industrial,19.6,21.6,20/06/2005,88.582960
15791,536,Assam,Tezpur,Residential,7.0,13.0,30/09/2015,NaN


In [3]:
y_train

14606    1
21850    0
23190    0
16552    0
14739    0
        ..
14654    1
1605     0
26844    0
15791    0
27775    0
Name: high_level, Length: 22296, dtype: int64

## Our first simple pipeline

Let's create a simple pipeline that impute missing values from <code>so2</code> and <code>no2</code>.

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [5]:
pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())
])

In [6]:
pipe.fit(X_train[['so2', 'no2']])

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())])

In [7]:
pipe.transform(X_test[['so2', 'no2']])

array([[-8.55861816e-01, -5.29957605e-01],
       [-6.80542349e-01, -6.01926507e-01],
       [-2.09947991e-01,  1.89731417e-01],
       ...,
       [ 1.63910492e-16, -1.17214165e+00],
       [-3.94494798e-01, -1.42432747e-01],
       [-7.63588413e-01, -2.53154135e-01]])

In [8]:
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(SimpleImputer(), StandardScaler())

pipe.fit(X_train[['so2', 'no2']])

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

## Visualizing pipelines in html
We can turn on the diagram visualization to have a nice view of our pipelines:

In [8]:
from sklearn import set_config

set_config(display='diagram')

pipe

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())])


<h2 id="FeatureUnion">FeatureUnion</h2>



<ul>
<li>Applies transformers in paralell, independently</li>
<li>Concatenate feature matrices outputs of each transformer</li>
<li>Usefull to create and add new features</li>
</ul>


In [9]:
from sklearn.pipeline import FeatureUnion

union = FeatureUnion([
    ('pipeline', pipe), # columns 0-1
    ('not_scaled', SimpleImputer()) # new colums 2-3
])

union.fit(X_train[['so2', 'no2']])
union



FeatureUnion(transformer_list=[('pipeline',
                                Pipeline(steps=[('imputer', SimpleImputer()),
                                                ('scaler', StandardScaler())])),
                               ('not_scaled', SimpleImputer())])

In [10]:
pd.DataFrame(union.transform(X_test[['so2', 'no2']])).head()

,0,1,2,3
0,-0.855862,-0.529958,2.0,17.0
1,-0.680542,-0.601927,3.9,15.7
2,-0.209948,0.189731,9.0,30.0
3,-0.117675,-0.585318,10.0,16.0
4,-0.671315,-0.972843,4.0,9.0


## Tackling more complex transformations using <code>Column Transformer</code>
* Apply specific changes to specific columns in **parallel**
* A <code>Pipeline</code> object can be passed in <code>ColumnTransformer</code> and vice-versa

<code>from sklearn.compose import ColumnTransformer</code>


💻 Let's do this<br>

* *impute* then *scale* numerical variables
* *encode* categorical variables

In [11]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Impute then Scale for numerical variables: 
num_transformer = Pipeline([
    ('num_imputer', SimpleImputer()),
    ('num_scaler', StandardScaler())])

# Encode categorical variables
cat_transformer = Pipeline([
    ('cat_imputer',SimpleImputer(strategy = 'most_frequent')),
    ('cat_encoder',OneHotEncoder(handle_unknown='ignore', sparse_output=False))
     ])

In [12]:
# Paralellize "num_transformer" and "cat_transformer"
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, X_train.select_dtypes(include=np.number).columns),
    ('cat_transformer', cat_transformer, X_train.select_dtypes(exclude=np.number).columns)
])

preprocessor

ColumnTransformer(transformers=[('num_transformer',
                                 Pipeline(steps=[('num_imputer',
                                                  SimpleImputer()),
                                                 ('num_scaler',
                                                  StandardScaler())]),
                                 Index(['so2', 'no2', 'rainfall'], dtype='object')),
                                ('cat_transformer',
                                 Pipeline(steps=[('cat_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('cat_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['stn_code', 'state', 'location', 'type', 'date'], dtype='object'))])

In [13]:
X_train.dtypes

stn_code     object
state        object
location     object
type         object
so2         float64
no2         float64
date         object
rainfall    float64
dtype: object

In [14]:
# Tip to easily select columns in a dataframe based on type:
X_train.select_dtypes(exclude=np.number).columns

Index(['stn_code', 'state', 'location', 'type', 'date'], dtype='object')

In [16]:
X_train.describe()

,so2,no2,rainfall
count,20824.000000,22296.000000,19409.000000
mean,11.275282,26.572814,80.111746
std,11.214122,18.063762,20.102804
min,0.000000,0.000000,-3.790493
25%,5.000000,14.900000,66.656679
50%,8.200000,23.000000,80.031149
75%,14.000000,33.000000,93.713891
max,363.000000,363.000000,158.895815


In [21]:
preprocessor.fit_transform(X_train)

array([[ 1.91234029,  1.4408831 ,  0.76581787, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.26642647, -0.03171136, -1.77332097, ...,  0.        ,
         0.        ,  0.        ],
       [-0.85586182, -0.91748246,  0.42619511, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.76815009, -0.27529841,  0.4516605 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.3944948 , -0.75140038,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.85586182, -1.19428593, -0.2981496 , ...,  0.        ,
         0.        ,  0.        ]])

## How about our feature names?

By default, `sklearn` transformers and pipelines return a numpy array. In the past, retaining the name of the columns was a bit of a juggling exercise. Luckily, starting with `sklearn version 1.2`, we can set the option of `sklearn` to return a `pandas` dataframe (<a href="https://blog.scikit-learn.org/technical/pandas-dataframe-output-for-sklearn-transformer/">See this post for explanation</a>):

In [22]:
from sklearn import set_config
set_config(transform_output = "pandas")

From now on, all of your object will return a pandas dataframe:

In [23]:
preprocessor.fit_transform(X_train)

,num_transformer__so2,num_transformer__no2,num_transformer__rainfall,cat_transformer__stn_code_1,cat_transformer__stn_code_10,cat_transformer__stn_code_101,cat_transformer__stn_code_102,cat_transformer__stn_code_103,cat_transformer__stn_code_106,cat_transformer__stn_code_109,...,cat_transformer__date_31/12/2005,cat_transformer__date_31/12/2007,cat_transformer__date_31/12/2008,cat_transformer__date_31/12/2009,cat_transformer__date_31/12/2010,cat_transformer__date_31/12/2011,cat_transformer__date_31/12/2012,cat_transformer__date_31/12/2013,cat_transformer__date_31/12/2014,cat_transformer__date_31/12/2015
14606,1.912340,1.440883,0.765818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21850,1.266426,-0.031711,-1.773321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23190,-0.855862,-0.917482,0.426195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16552,1.912340,-0.529958,-0.474065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14739,0.159146,-0.751400,1.619155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14654,-0.486768,0.189731,0.625940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1605,0.020736,-0.590854,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26844,0.768150,-0.275298,0.451661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15791,-0.394495,-0.751400,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Makes our life much easier!
But there are limitations. This won't work if you need a sparse output. For this, simply use `transform_output = "default"`.

### Why did we get >5000 features?


* We selected **ALL** non-numeric values for the OneHotEncoder

* This includes the date, station, location columns!

* Each individual date, station and location become a feature column: this leads to an explosion of features. Let's fix this by selecting only the columns we want.

In [25]:
# Paralellize "num_transformer" and "cat_transformer"
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, X_train.select_dtypes(include=np.number).columns),
    ('cat_transformer', cat_transformer, ['state', 'type'])
])

preprocessor

ColumnTransformer(transformers=[('num_transformer',
                                 Pipeline(steps=[('num_imputer',
                                                  SimpleImputer()),
                                                 ('num_scaler',
                                                  StandardScaler())]),
                                 Index(['so2', 'no2', 'rainfall'], dtype='object')),
                                ('cat_transformer',
                                 Pipeline(steps=[('cat_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('cat_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['state', 'type'])])

In [26]:
preprocessor.fit_transform(X_train)

,num_transformer__so2,num_transformer__no2,num_transformer__rainfall,cat_transformer__state_Andhra Pradesh,cat_transformer__state_Arunachal Pradesh,cat_transformer__state_Assam,cat_transformer__state_Bihar,cat_transformer__state_Chandigarh,cat_transformer__state_Chhattisgarh,cat_transformer__state_Dadra & Nagar Haveli,...,cat_transformer__state_Tamil Nadu,cat_transformer__state_Telangana,cat_transformer__state_Uttar Pradesh,cat_transformer__state_Uttarakhand,cat_transformer__state_Uttaranchal,cat_transformer__state_West Bengal,cat_transformer__type_Industrial,cat_transformer__type_Others,cat_transformer__type_Residential,cat_transformer__type_Sensitive
14606,1.912340,1.440883,0.765818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
21850,1.266426,-0.031711,-1.773321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
23190,-0.855862,-0.917482,0.426195,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
16552,1.912340,-0.529958,-0.474065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
14739,0.159146,-0.751400,1.619155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14654,-0.486768,0.189731,0.625940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1605,0.020736,-0.590854,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
26844,0.768150,-0.275298,0.451661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
15791,-0.394495,-0.751400,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [27]:
preprocessor.transform(X_test)

,num_transformer__so2,num_transformer__no2,num_transformer__rainfall,cat_transformer__state_Andhra Pradesh,cat_transformer__state_Arunachal Pradesh,cat_transformer__state_Assam,cat_transformer__state_Bihar,cat_transformer__state_Chandigarh,cat_transformer__state_Chhattisgarh,cat_transformer__state_Dadra & Nagar Haveli,...,cat_transformer__state_Tamil Nadu,cat_transformer__state_Telangana,cat_transformer__state_Uttar Pradesh,cat_transformer__state_Uttarakhand,cat_transformer__state_Uttaranchal,cat_transformer__state_West Bengal,cat_transformer__type_Industrial,cat_transformer__type_Others,cat_transformer__type_Residential,cat_transformer__type_Sensitive
12002,-8.558618e-01,-0.529958,0.372909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4305,-6.805423e-01,-0.601927,-0.875823,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9394,-2.099480e-01,0.189731,-0.283205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
16223,-1.176746e-01,-0.585318,3.192687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21271,-6.713150e-01,-0.972843,-1.061826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16756,1.266426e+00,0.189731,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8915,-8.558618e-01,-1.138925,0.638040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
18188,1.639105e-16,-1.172142,-0.502512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
16070,-3.944948e-01,-0.142433,0.041890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Accessing individual transformers

We can easily access individual transformers in the pipeline, and see their properties:

In [28]:
preprocessor.transformers_

[('num_transformer',
  Pipeline(steps=[('num_imputer', SimpleImputer()),
                  ('num_scaler', StandardScaler())]),
  Index(['so2', 'no2', 'rainfall'], dtype='object')),
 ('cat_transformer',
  Pipeline(steps=[('cat_imputer', SimpleImputer(strategy='most_frequent')),
                  ('cat_encoder',
                   OneHotEncoder(handle_unknown='ignore', sparse_output=False))]),
  ['state', 'type']),
 ('remainder', 'drop', [0, 2, 6])]

In [21]:
preprocessor.transformers_[0][2]

Index(['so2', 'no2', 'rainfall'], dtype='object')

In [22]:
preprocessor.transformers_[1][1][1].get_feature_names_out()

array(['state_Andhra Pradesh', 'state_Arunachal Pradesh', 'state_Assam',
       'state_Bihar', 'state_Chandigarh', 'state_Chhattisgarh',
       'state_Dadra & Nagar Haveli', 'state_Daman & Diu', 'state_Delhi',
       'state_Goa', 'state_Gujarat', 'state_Haryana',
       'state_Himachal Pradesh', 'state_Jammu & Kashmir',
       'state_Jharkhand', 'state_Karnataka', 'state_Kerala',
       'state_Madhya Pradesh', 'state_Maharashtra', 'state_Manipur',
       'state_Meghalaya', 'state_Mizoram', 'state_Nagaland',
       'state_Odisha', 'state_Puducherry', 'state_Punjab',
       'state_Rajasthan', 'state_Tamil Nadu', 'state_Telangana',
       'state_Uttar Pradesh', 'state_Uttarakhand', 'state_Uttaranchal',
       'state_West Bengal', 'type_Industrial', 'type_Others',
       'type_Residential', 'type_Sensitive'], dtype=object)


<h3 id="Custom-transformer-(basic)">Custom transformer (basic)<a class="anchor-link" href="https://kitt.lewagon.com/karr/data-lectures.kitt/05-ML_08-Workflow.html?title=Workflow&amp;program_id=10#Custom-transformer-(basic)">¶</a></h3><div class="highlight"><pre><span></span><span class="kn">from</span> <span class="nn">sklearn.preprocessing</span> <span class="kn">import</span> <span class="n">FunctionTransformer</span>
</pre></div>
<ul>
<li><p>You will often need to perform custom transformations on your columns</p>
</li>
<li><p><code>FunctionTransformer</code> encapsulates a function into a transformer object</p>
</li>
<li><p>Can work with Pipelines (in series) or with ColumnTransformer (in paralell)</p>
</li>
</ul>


In [29]:
from sklearn.preprocessing import FunctionTransformer

# Create a transformer that compresses data to 2 digits (for instance!)
rounder = FunctionTransformer(np.round)
# rounder = FunctionTransformer(lambda array: np.round(array, decimals=2))

# Add it at the end of our numerical transformer
num_transformer = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('rounder', rounder)])

cat_transformer = Pipeline([
    ('cat_imputer',SimpleImputer(strategy = 'most_frequent')),
    ('cat_encoder',OneHotEncoder(handle_unknown='ignore', sparse_output=False))
     ])

preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, X_train.select_dtypes(include=np.number).columns),
    ('cat_transformer', cat_transformer, ['state', 'type'])
])

preprocessor.fit_transform(X_train)

,num_transformer__so2,num_transformer__no2,num_transformer__rainfall,cat_transformer__state_Andhra Pradesh,cat_transformer__state_Arunachal Pradesh,cat_transformer__state_Assam,cat_transformer__state_Bihar,cat_transformer__state_Chandigarh,cat_transformer__state_Chhattisgarh,cat_transformer__state_Dadra & Nagar Haveli,...,cat_transformer__state_Tamil Nadu,cat_transformer__state_Telangana,cat_transformer__state_Uttar Pradesh,cat_transformer__state_Uttarakhand,cat_transformer__state_Uttaranchal,cat_transformer__state_West Bengal,cat_transformer__type_Industrial,cat_transformer__type_Others,cat_transformer__type_Residential,cat_transformer__type_Sensitive
14606,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
21850,1.0,-0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
23190,-1.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
16552,2.0,-1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
14739,0.0,-1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14654,-0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1605,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
26844,1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
15791,-0.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [30]:
preprocessor

ColumnTransformer(transformers=[('num_transformer',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler()),
                                                 ('rounder',
                                                  FunctionTransformer(func=<function round at 0x106de05f0>))]),
                                 Index(['so2', 'no2', 'rainfall'], dtype='object')),
                                ('cat_transformer',
                                 Pipeline(steps=[('cat_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('cat_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['state', 'type'])])


<p>⚠️ <code>FunctionTransformer</code> only works for <strong>stateless</strong> transformations</p>
<ul>
<li>It cannot "store" information on a <code>fit</code> (for instance on a train set)</li>
<li>In order to apply it back later on a <code>transform</code> (for instance on the test set)</li>
</ul>
<p>✅ stateless transformations</p>
<p><span class="MathJax_Preview" style="color: inherit;"></span><span class="mjx-chtml MathJax_CHTML" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;X&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-3-Frame" role="presentation" style="font-size: 116%; position: relative;" tabindex="0"><span aria-hidden="true" class="mjx-math" id="MJXc-Node-7"><span class="mjx-mrow" id="MJXc-Node-8"><span class="mjx-mi" id="MJXc-Node-9"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.249em; padding-right: 0.024em;">X</span></span></span></span><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>X</mi></math></span></span><script id="MathJax-Element-3" type="math/tex">X</script> --&gt; <span class="MathJax_Preview" style="color: inherit;"></span><span class="mjx-chtml MathJax_CHTML" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;l&lt;/mi&gt;&lt;mi&gt;o&lt;/mi&gt;&lt;mi&gt;g&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;X&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-4-Frame" role="presentation" style="font-size: 116%; position: relative;" tabindex="0"><span aria-hidden="true" class="mjx-math" id="MJXc-Node-10"><span class="mjx-mrow" id="MJXc-Node-11"><span class="mjx-mi" id="MJXc-Node-12"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.496em; padding-bottom: 0.311em;">l</span></span><span class="mjx-mi" id="MJXc-Node-13"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.249em; padding-bottom: 0.311em;">o</span></span><span class="mjx-mi" id="MJXc-Node-14"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.249em; padding-bottom: 0.496em; padding-right: 0.003em;">g</span></span><span class="mjx-mo" id="MJXc-Node-15"><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.434em; padding-bottom: 0.619em;">(</span></span><span class="mjx-mi" id="MJXc-Node-16"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.249em; padding-right: 0.024em;">X</span></span><span class="mjx-mo" id="MJXc-Node-17"><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.434em; padding-bottom: 0.619em;">)</span></span></span></span><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>l</mi><mi>o</mi><mi>g</mi><mo stretchy="false">(</mo><mi>X</mi><mo stretchy="false">)</mo></math></span></span><script id="MathJax-Element-4" type="math/tex">log(X)</script><br/>
<span class="MathJax_Preview" style="color: inherit;"></span><span class="mjx-chtml MathJax_CHTML" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;X&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;/msub&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;X&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;/msub&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-5-Frame" role="presentation" style="font-size: 116%; position: relative;" tabindex="0"><span aria-hidden="true" class="mjx-math" id="MJXc-Node-18"><span class="mjx-mrow" id="MJXc-Node-19"><span class="mjx-mo" id="MJXc-Node-20"><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.434em; padding-bottom: 0.619em;">(</span></span><span class="mjx-msubsup" id="MJXc-Node-21"><span class="mjx-base" style="margin-right: -0.024em;"><span class="mjx-mi" id="MJXc-Node-22"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.249em; padding-right: 0.024em;">X</span></span></span><span class="mjx-sub" style="font-size: 70.7%; vertical-align: -0.212em; padding-right: 0.071em;"><span class="mjx-mn" id="MJXc-Node-23" style=""><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.373em; padding-bottom: 0.373em;">1</span></span></span></span><span class="mjx-mo" id="MJXc-Node-24"><span class="mjx-char MJXc-TeX-main-R" style="margin-top: -0.182em; padding-bottom: 0.557em;">,</span></span><span class="mjx-msubsup MJXc-space1" id="MJXc-Node-25"><span class="mjx-base" style="margin-right: -0.024em;"><span class="mjx-mi" id="MJXc-Node-26"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.249em; padding-right: 0.024em;">X</span></span></span><span class="mjx-sub" style="font-size: 70.7%; vertical-align: -0.212em; padding-right: 0.071em;"><span class="mjx-mn" id="MJXc-Node-27" style=""><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.373em; padding-bottom: 0.373em;">2</span></span></span></span><span class="mjx-mo" id="MJXc-Node-28"><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.434em; padding-bottom: 0.619em;">)</span></span></span></span><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mo stretchy="false">(</mo><msub><mi>X</mi><mn>1</mn></msub><mo>,</mo><msub><mi>X</mi><mn>2</mn></msub><mo stretchy="false">)</mo></math></span></span><script id="MathJax-Element-5" type="math/tex">(X_1, X_2)</script> --&gt; <span class="MathJax_Preview" style="color: inherit;"></span><span class="mjx-chtml MathJax_CHTML" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;X&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mn&gt;5&lt;/mn&gt;&lt;msub&gt;&lt;mi&gt;X&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-6-Frame" role="presentation" style="font-size: 116%; position: relative;" tabindex="0"><span aria-hidden="true" class="mjx-math" id="MJXc-Node-29"><span class="mjx-mrow" id="MJXc-Node-30"><span class="mjx-msubsup" id="MJXc-Node-31"><span class="mjx-base" style="margin-right: -0.024em;"><span class="mjx-mi" id="MJXc-Node-32"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.249em; padding-right: 0.024em;">X</span></span></span><span class="mjx-sub" style="font-size: 70.7%; vertical-align: -0.212em; padding-right: 0.071em;"><span class="mjx-mn" id="MJXc-Node-33" style=""><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.373em; padding-bottom: 0.373em;">1</span></span></span></span><span class="mjx-mo MJXc-space2" id="MJXc-Node-34"><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.311em; padding-bottom: 0.434em;">+</span></span><span class="mjx-mn MJXc-space2" id="MJXc-Node-35"><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.373em; padding-bottom: 0.373em;">5</span></span><span class="mjx-msubsup" id="MJXc-Node-36"><span class="mjx-base" style="margin-right: -0.024em;"><span class="mjx-mi" id="MJXc-Node-37"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.249em; padding-right: 0.024em;">X</span></span></span><span class="mjx-sub" style="font-size: 70.7%; vertical-align: -0.212em; padding-right: 0.071em;"><span class="mjx-mn" id="MJXc-Node-38" style=""><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.373em; padding-bottom: 0.373em;">2</span></span></span></span></span></span><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>X</mi><mn>1</mn></msub><mo>+</mo><mn>5</mn><msub><mi>X</mi><mn>2</mn></msub></math></span></span><script id="MathJax-Element-6" type="math/tex">X_1 + 5X_2</script></p>
<p>❌ Memory-dependent transformation</p>
<p><span class="MathJax_Preview" style="color: inherit;"></span><span class="mjx-chtml MathJax_CHTML" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;X&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-7-Frame" role="presentation" style="font-size: 116%; position: relative;" tabindex="0"><span aria-hidden="true" class="mjx-math" id="MJXc-Node-39"><span class="mjx-mrow" id="MJXc-Node-40"><span class="mjx-mi" id="MJXc-Node-41"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.249em; padding-right: 0.024em;">X</span></span></span></span><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>X</mi></math></span></span><script id="MathJax-Element-7" type="math/tex">X</script> --&gt; <span class="MathJax_Preview" style="color: inherit;"></span><span class="mjx-chtml MathJax_CHTML" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;S&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mi&gt;S&lt;/mi&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mi&gt;l&lt;/mi&gt;&lt;mi&gt;e&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;X&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-8-Frame" role="presentation" style="font-size: 116%; position: relative;" tabindex="0"><span aria-hidden="true" class="mjx-math" id="MJXc-Node-42"><span class="mjx-mrow" id="MJXc-Node-43"><span class="mjx-mi" id="MJXc-Node-44"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.496em; padding-bottom: 0.311em; padding-right: 0.032em;">S</span></span><span class="mjx-mi" id="MJXc-Node-45"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.311em;">t</span></span><span class="mjx-mi" id="MJXc-Node-46"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.249em; padding-bottom: 0.311em;">a</span></span><span class="mjx-mi" id="MJXc-Node-47"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.249em; padding-bottom: 0.311em;">n</span></span><span class="mjx-mi" id="MJXc-Node-48"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.496em; padding-bottom: 0.311em; padding-right: 0.003em;">d</span></span><span class="mjx-mi" id="MJXc-Node-49"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.249em; padding-bottom: 0.311em;">a</span></span><span class="mjx-mi" id="MJXc-Node-50"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.249em; padding-bottom: 0.311em;">r</span></span><span class="mjx-mi" id="MJXc-Node-51"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.496em; padding-bottom: 0.311em; padding-right: 0.003em;">d</span></span><span class="mjx-mi" id="MJXc-Node-52"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.496em; padding-bottom: 0.311em; padding-right: 0.032em;">S</span></span><span class="mjx-mi" id="MJXc-Node-53"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.249em; padding-bottom: 0.311em;">c</span></span><span class="mjx-mi" id="MJXc-Node-54"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.249em; padding-bottom: 0.311em;">a</span></span><span class="mjx-mi" id="MJXc-Node-55"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.496em; padding-bottom: 0.311em;">l</span></span><span class="mjx-mi" id="MJXc-Node-56"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.249em; padding-bottom: 0.311em;">e</span></span><span class="mjx-mi" id="MJXc-Node-57"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.249em; padding-bottom: 0.311em;">r</span></span><span class="mjx-mo" id="MJXc-Node-58"><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.434em; padding-bottom: 0.619em;">(</span></span><span class="mjx-mi" id="MJXc-Node-59"><span class="mjx-char MJXc-TeX-math-I" style="padding-top: 0.434em; padding-bottom: 0.249em; padding-right: 0.024em;">X</span></span><span class="mjx-mo" id="MJXc-Node-60"><span class="mjx-char MJXc-TeX-main-R" style="padding-top: 0.434em; padding-bottom: 0.619em;">)</span></span></span></span><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>S</mi><mi>t</mi><mi>a</mi><mi>n</mi><mi>d</mi><mi>a</mi><mi>r</mi><mi>d</mi><mi>S</mi><mi>c</mi><mi>a</mi><mi>l</mi><mi>e</mi><mi>r</mi><mo stretchy="false">(</mo><mi>X</mi><mo stretchy="false">)</mo></math></span></span><script id="MathJax-Element-8" type="math/tex">StandardScaler(X)</script></p>
<p>☝️ For this, we will have to code our own <code>Class</code></p>



<h3 id="Custom-transformer-(advanced)">Custom transformer (advanced)<a class="anchor-link" href="https://kitt.lewagon.com/karr/data-lectures.kitt/05-ML_08-Workflow.html?title=Workflow&amp;program_id=10#Custom-transformer-(advanced)">¶</a></h3><ul>
<li>To memorize variables (e.g. column mean etc...) during the <code>.fit()</code></li>
<li>To reuse later these variables with <code>.transform()</code> on a different dataset</li>
<li>Must create a Class with both methods and storing memory as <em>instance variables</em></li>
</ul>



<p>✏️ Let's code a <code>CustomScaler(shrink_factor=3)</code> that center data around its mean, and shrink it by a <em>fixed</em> factor</p>
<ul>
<li>we will only <code>fit</code> it on the <code>X_train</code></li>
<li>we will use it to <code>transform</code> the <code>X_test</code></li>
</ul>


In [25]:

from sklearn.base import TransformerMixin, BaseEstimator

class CustomScaler(TransformerMixin, BaseEstimator): 
# TransformerMixin generates a fit_transform method from fit and transform
# BaseEstimator generates get_params and set_params methods

    def __init__(self, shrink_factor=3):
        self.shrink_factor = shrink_factor

    def fit(self, X, y=None):
        self.means = X.mean()
        return self

    def transform(self, X, y=None):
        X_transformed = (X - self.means) / self.shrink_factor
        # Return result as dataframe for integration into ColumnTransformer
        return X_transformed



In [27]:

# The CustomScaler can then be used like any other transformer!
custom_scaler = CustomScaler(shrink_factor=3)
custom_scaler.fit(X_train[['no2','rainfall','so2']])
custom_scaler.transform(X_test[['no2','rainfall','so2']]).head()



,no2,rainfall,so2
25679,0.82276,6.090122,0.920359
18215,-4.17724,1.213941,-1.746308
13730,-0.67724,-0.565268,1.287026
4319,-6.67724,0.558198,NaN
24151,-0.17724,-0.481491,0.587026


In [28]:
preprocessor.transformers_

[('num_transformer',
  Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                  ('rounder',
                   FunctionTransformer(func=<function <lambda> at 0x17b649d00>))]),
  Index(['so2', 'no2', 'rainfall'], dtype='object')),
 ('cat_transformer',
  Pipeline(steps=[('cat_imputer', SimpleImputer(strategy='most_frequent')),
                  ('cat_encoder',
                   OneHotEncoder(handle_unknown='ignore', sparse_output=False))]),
  ['state', 'type']),
 ('remainder', 'drop', [0, 2, 6])]

In [29]:
preprocessor.transformers_[2] = ('custom_scaler', CustomScaler(shrink_factor=3), X_train.select_dtypes(include=np.number))

In [30]:
preprocessor.fit_transform(X_train)

,num_transformer__so2,num_transformer__no2,num_transformer__rainfall,cat_transformer__state_Andhra Pradesh,cat_transformer__state_Arunachal Pradesh,cat_transformer__state_Assam,cat_transformer__state_Bihar,cat_transformer__state_Chandigarh,cat_transformer__state_Chhattisgarh,cat_transformer__state_Dadra & Nagar Haveli,...,cat_transformer__state_Tamil Nadu,cat_transformer__state_Telangana,cat_transformer__state_Uttar Pradesh,cat_transformer__state_Uttarakhand,cat_transformer__state_Uttaranchal,cat_transformer__state_West Bengal,cat_transformer__type_Industrial,cat_transformer__type_Others,cat_transformer__type_Residential,cat_transformer__type_Sensitive
19849,0.37,-0.31,1.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7683,-0.90,-1.18,0.77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1909,4.87,1.85,0.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
27450,-0.66,-1.05,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8266,-0.90,-0.53,-0.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14912,-0.02,-0.47,1.21,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
18788,1.15,0.03,0.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10756,-0.71,-0.36,0.64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
25022,-0.07,0.18,1.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [31]:
preprocessor

ColumnTransformer(transformers=[('num_transformer',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler()),
                                                 ('rounder',
                                                  FunctionTransformer(func=<function <lambda> at 0x17b649d00>))]),
                                 Index(['so2', 'no2', 'rainfall'], dtype='object')),
                                ('cat_transformer',
                                 Pipeline(steps=[('cat_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('cat_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['state', 'type'])])

# Including models in pipelines
<br>

<center><img src="https://drive.google.com/uc?id=145ST_iBDPTifgO5F7slXyJsOrmgCRpmr" style="width:900px;"><br>
 © Cédric John, 2022; Image generated with <a href="https://openai.com/blog/dall-e/">DALL-E</a><br>
<br>Prompt: Dramatic view of a rusted yellow oil pump from the 1960's surrounded by the streets of Havana, Cuba.</center>


* Model objects can be plugged into pipelines
* Pipelines inherit the methods of the **last** object in the sequence
* Transformers: <code>fit</code> and <code>transform</code>
* Models: <code>fit</code>, <code>score</code>, <code>predict</code>, etc...


<img src="https://drive.google.com/uc?id=13pBbXtuoOyHn8NlZhbsdBVk-kVnib-ep" style="width:1200px;">
<a href="https://www.packtpub.com/product/python-machine-learning-third-edition/9781789955750">Rashka, S., 2015, Packt Publishing</a>

In [32]:
from sklearn.linear_model import LogisticRegression

# Combine preprocessor and linear model in pipeline
final_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression(max_iter=2500))])
final_pipe


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('rounder',
                                                                   FunctionTransformer(func=<function round at 0x106de05f0>))]),
                                                  Index(['so2', 'no2', 'rainfall'], dtype='object')),
                                                 ('cat_transformer',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('cat_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['state', 'type'])])),
                ('classifier', LogisticRegression(max_iter=2500))])

In [33]:
# Train pipeline
final_pipe_trained = final_pipe.fit(X_train, y_train)

In [34]:
# Make predictions
final_pipe_trained.predict(X_test)

array([0, 0, 1, ..., 0, 1, 1])

In [35]:
# Score model
final_pipe_trained.score(X_test, y_test)

0.71654108360244

### Cross validate a pipeline

In [36]:

from sklearn.model_selection import cross_val_score

# Cross validate pipeline
cross_val_score(final_pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.7203083801894089


<h3 id="Test-your-pipeline-as-you-build-it">Test your pipeline as you build it<a class="anchor-link" href="https://kitt.lewagon.com/karr/data-lectures.kitt/05-ML_08-Workflow.html?title=Workflow&amp;program_id=10#Test-your-pipeline-as-you-build-it">¶</a></h3><p>As you are building your pipeline, it is important to ensure it works identically to what you have done in your notebook thus far.</p>
<ul>
<li>Check the data preprocessing: Compare the statistics of preprocessed data out of the pipeline to the ones of the same data preprocessed outside the pipeline</li>
<li>Compare the performance of the model out of the pipeline to the one trained outside the pipeline</li>
</ul>


In [37]:
# Access component of pipeline with `name_steps`
final_pipe.named_steps["preprocessing"].fit_transform(X_train).shape

(22296, 40)

## Exporting models and Pipelines

<li>You can export your final model/pipeline as a pickle file</li>
<li>The file can be loaded back into a notebook or deployed on a server</li>
</ul>


In [38]:
# LET'S START WITH A CLEAN PIPELINE
num_transformer = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())])

# Paralellize "num_transformer" and "cat_transformer"
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, X_train.select_dtypes(include=np.number).columns),
    ('cat_transformer', cat_transformer, ['state', 'type'])
])

# Combine preprocessor and linear model in pipeline
final_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression(max_iter=2500))])

final_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['so2', 'no2', 'rainfall'], dtype='object')),
                                                 ('cat_transformer',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('cat_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['state', 'type'])])),
                ('classifier', LogisticRegression(max_iter=2500))])

In [39]:
import pickle

# Export pipeline as pickle file
with open("pipeline.pkl", "wb") as file:
    pickle.dump(final_pipe, file)

In [40]:
# Load pipeline from pickle file
my_pipeline = pickle.load(open("pipeline.pkl", "rb"))
new_samples = pd.read_csv('Lecture_data/new_samples.csv')
new_samples.head()

,stn_code,state,location,type,so2,no2,date,rainfall
0,527,Madhya Pradesh,Ujjain,Industrial,7.8,9.1,13/10/2011,98.903760
1,258,Uttar Pradesh,Ghaziabad,Industrial,35.0,39.0,24/05/2011,126.244780
2,638,Maharashtra,Chandrapur,Industrial,15.0,10.0,21/07/2011,105.816791
3,537,Assam,Sivasagar,Residential,6.0,14.5,08/05/2013,49.449315
4,376,Rajasthan,Jodhpur,Residential,4.0,19.0,23/07/2004,86.035515


In [40]:
new_samples = pd.read_csv('Lecture_data/new_samples.csv')
new_samples.head()

,stn_code,state,location,type,so2,no2,date,rainfall
0,527,Madhya Pradesh,Ujjain,Industrial,7.8,9.1,13/10/2011,98.903760
1,258,Uttar Pradesh,Ghaziabad,Industrial,35.0,39.0,24/05/2011,126.244780
2,638,Maharashtra,Chandrapur,Industrial,15.0,10.0,21/07/2011,105.816791
3,537,Assam,Sivasagar,Residential,6.0,14.5,08/05/2013,49.449315
4,376,Rajasthan,Jodhpur,Residential,4.0,19.0,23/07/2004,86.035515


In [41]:
pred = my_pipeline.predict(new_samples)
new_samples['Predictions'] = pred

pred

array([1, 1, 0, ..., 1, 1, 0])

In [41]:
pred = final_pipe.predict(new_samples)
pred

array([1, 1, 0, ..., 1, 1, 0])

# Moving from Notebooks to Packages
<br>

<center><img src="https://drive.google.com/uc?id=13sDCVeMXEc3kXqI1F_udT8SzAHaEYLqY" style="width:900px;"><br>
 © Cédric John, 2022; Image generated with <a href="https://openai.com/blog/dall-e/">DALL-E</a><br>
<br>Prompt: 35 view of a well-organised giant warehouse with endless shelves full of items, orange lighting.</center>


<p><img src="https://drive.google.com/uc?id=149_jQxru5A1q15RiF9860uatiBDRUfmz"/></p>



<p><img src="https://drive.google.com/uc?id=145bhQK2Gkcv9d7FgFG8rxyPCMOAW5dNz" width="1600"/></p>



<h3 id="Notebooks">Notebooks<a class="anchor-link" href="https://kitt.lewagon.com/karr/data-lectures.kitt/07-Data-Engineering_01.html?title=Code+as+a+Product&amp;program_id=10#Notebooks">¶</a></h3><p>👉 Great for exploration</p>
<p>👉 Offer visible code feedback</p>
<p>👉 Plots and graphs</p>



<h3 id="Packages">Classes / Packages<a class="anchor-link" href="https://kitt.lewagon.com/karr/data-lectures.kitt/07-Data-Engineering_01.html?title=Code+as+a+Product&amp;program_id=10#Packages">¶</a></h3><p>👉 Reusable code</p>
<p>Reuse your code from one project to another, share it with your colleagues, or open source it (remember <code>pip</code>?)</p>



<p>👉 Deployable code</p>
<p>Packages are the python standard to exchange code, they will allows us to run our code online and on-demand</p>



<p>👉 Automation of tests and deployment (CI/CD)</p>
<p>We want our code to be automatically validated, deployed to production, and to run without error</p>
<p>We want to make sure our newly developed features do not break existing ones</p>


# From Notebook to Python Class


## Given our exploratory notebook:

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Load and split data
data = pd.read_csv('Lecture_data/India_air_quality_light.csv')
data = data.drop_duplicates()

y = data.high_level # Target
X = data.drop(columns='high_level') #Feature

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

# Preprocess data

# Impute then Scale for numerical variables: 
num_transformer = Pipeline([
    ('num_imputer', SimpleImputer()),
    ('num_scaler', StandardScaler())])

# Encode categorical variables
cat_transformer = Pipeline([
    ('cat_imputer',SimpleImputer(strategy = 'most_frequent')),
    ('cat_encoder',OneHotEncoder(handle_unknown='ignore', sparse_output=False))
     ])

preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, X_train.select_dtypes(include=np.number).columns),
    ('cat_transformer', cat_transformer, ['state', 'type'])
])

final_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression(max_iter=2500))])


# Train model with grid search
final_pipe.fit(X_train, y_train)

# Score final model
final_pipe.score(X_test, y_test)

# Predict (new) samples
new_samples = pd.read_csv('Lecture_data/new_samples.csv')
pred = final_pipe.predict(new_samples)
new_samples['Predictions'] = pred

new_samples.head()


,stn_code,state,location,type,so2,no2,date,rainfall,Predictions
0,527,Madhya Pradesh,Ujjain,Industrial,7.8,9.1,13/10/2011,98.903760,1
1,258,Uttar Pradesh,Ghaziabad,Industrial,35.0,39.0,24/05/2011,126.244780,1
2,638,Maharashtra,Chandrapur,Industrial,15.0,10.0,21/07/2011,105.816791,0
3,537,Assam,Sivasagar,Residential,6.0,14.5,08/05/2013,49.449315,0
4,376,Rajasthan,Jodhpur,Residential,4.0,19.0,23/07/2004,86.035515,1


## Let's create a class that does the following:
* Train the best Logistics Regression model and saves it
* Prints our test score
* Load the model and new data to make predictions

## There are multiple ways to do this:
* We can write one <code>PollutionModel</code> class that does it all
* We can write one <code>TrainPollutionModel</code> for training and one <code>PredictPollution</code> class
* We can write more classes as well as files containing utility functions
* ...

👉🏽 Because our code is simple, let's do one class only

## Let's try our new class!

In [46]:
from pollution_model import PollutionModel

model = PollutionModel('https://drive.google.com/uc?id=13nr-VHGi2zFRpfHhCfz-yHJLX3LQrfTX')

In [47]:
model.score_model()

0.7305346250448511

In [48]:
model.predict('https://drive.google.com/uc?id=13k22u8G6FvV7f1Z8rsCrLksLY3j6SNo1')

,stn_code,state,location,type,so2,no2,date,rainfall,Predictions
0,527,Madhya Pradesh,Ujjain,Industrial,7.8,9.1,13/10/2011,98.903760,1
1,258,Uttar Pradesh,Ghaziabad,Industrial,35.0,39.0,24/05/2011,126.244780,1
2,638,Maharashtra,Chandrapur,Industrial,15.0,10.0,21/07/2011,105.816791,0
3,537,Assam,Sivasagar,Residential,6.0,14.5,08/05/2013,49.449315,0
4,376,Rajasthan,Jodhpur,Residential,4.0,19.0,23/07/2004,86.035515,1
...,...,...,...,...,...,...,...,...,...
30999,457,Karnataka,Bangalore,Residential,14.0,28.0,19/05/2014,NaN,0
31000,565,Himachal Pradesh,Nalagarh,Residential,1.0,10.0,17/01/2011,NaN,0
31001,574,Maharashtra,Sangli,Residential,14.0,36.0,21/03/2011,131.366458,1
31002,NaN,Maharashtra,Pune,Others,18.0,49.3,16/11/2007,94.813520,1


# Suggested Resources

## 📺 Videos 
#### Short videos from my Undegraduate Machine Learning Classes:
* 📼 <a href="https://youtu.be/VSGTHhUqIk4?list=PLZzjCZ3QdgQCcRIwQdd-_cJNAUgiEBB_n">Data preparation pipelines</a>

#### Others:

* 📼 <a href="https://developers.google.com/machine-learning/guides/rules-of-ml">Rules of Machine Learning</a> by Google Developers (gives advise on designing ML systems) 

## 📚 Further Reading 
* 📖 <a href="https://hazelcast.com/glossary/data-pipeline/">Data Pipelines</a> by Hazelcast
* 📖 <a href="https://python-packaging-tutorial.readthedocs.io/en/latest/setup_py.html">Python packages and easy introduction</a> 
* 📖 <a href="https://packaging.python.org/en/latest/tutorials/packaging-projects/">Packaging Python Projects</a> 


## 💻🐍 Time to Code ! 